# Facial Recognition - Minimal Version (For Memorization)

Binary classification: Modi vs Trump using OpenCV + CNN


In [ ]:
# Imports
import numpy as np
import cv2
import os
import glob
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt

np.random.seed(42)
tf.random.set_seed(42)


In [ ]:
# Load image paths
base_path = 'Assignmentdl2'
modi_images = glob.glob(os.path.join(base_path, 'modi', '*.jpg'))
trump_images = glob.glob(os.path.join(base_path, 'trump', '*.jpg'))

print(f"Modi: {len(modi_images)}, Trump: {len(trump_images)}")


In [ ]:
# Face detection setup
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def detect_and_extract_face(image_path, target_size=(128, 128)):
    img = cv2.imread(image_path)
    if img is None:
        return None
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    (x, y, w, h) = faces[0]
    face_roi = gray[y:y+h, x:x+w]
    face_resized = cv2.resize(face_roi, target_size)
    face_normalized = face_resized.astype('float32') / 255.0
    return face_normalized


In [ ]:
# Process all images
X = []
y = []

for img_path in modi_images:
    face = detect_and_extract_face(img_path)
    if face is not None:
        X.append(face)
        y.append(0)  # Modi = 0

for img_path in trump_images:
    face = detect_and_extract_face(img_path)
    if face is not None:
        X.append(face)
        y.append(1)  # Trump = 1

X = np.array(X)
y = np.array(y)
X = X.reshape(X.shape[0], X.shape[1], X.shape[2], 1)  # Add channel dimension

print(f"X shape: {X.shape}, y shape: {y.shape}")


In [ ]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print(f"Train: {X_train.shape}, Test: {X_test.shape}")


In [ ]:
# Build CNN model
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.summary()


In [ ]:
# Compile model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# Train model
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2, verbose=1)


In [ ]:
# Evaluate model
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f"Test Accuracy: {test_accuracy:.4f}")

# Make predictions
y_pred_proba = model.predict(X_test)
y_pred = (y_pred_proba >= 0.5).astype(int).flatten()

# Classification report
print(classification_report(y_test, y_pred, target_names=['Modi', 'Trump']))


In [ ]:
# Plot training history
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train')
plt.plot(history.history['val_accuracy'], label='Val')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train')
plt.plot(history.history['val_loss'], label='Val')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6, 5))
plt.imshow(cm, cmap='Blues')
plt.colorbar()
plt.xticks([0, 1], ['Modi', 'Trump'])
plt.yticks([0, 1], ['Modi', 'Trump'])
for i in range(2):
    for j in range(2):
        plt.text(j, i, cm[i, j], ha='center', va='center', fontsize=14, fontweight='bold')
plt.ylabel('True')
plt.xlabel('Predicted')
plt.title('Confusion Matrix')
plt.tight_layout()
plt.show()


In [ ]:
# Save model
model.save('facial_recognition_model.h5')
print("Model saved!")
